In [6]:
import shlex, subprocess, io, sys, re, csv
from importlib import reload
reload(sys)
from textblob import TextBlob

In [7]:
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])| (\w +:\ / \ / \S +)", " ", tweet).split())

In [8]:
def SentiStrength(tweet):
    p = subprocess.Popen(shlex.split("java -jar SentiStrength.jar stdin sentidata SentStrength_Data/ binary"), 
                         stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    try:
        stdout_text, stderr_text = p.communicate(tweet.replace(" ", "+").encode())
        #print stdout_text, stderr_text
    except UnicodeDecodeError:
        return "false"
    stdout_text = stdout_text.rstrip().decode().replace("\t","")
    return stdout_text

In [9]:
count = 0
f_input= open("tweet_input.csv",encoding="utf8")
f_output= open("nltk_output.txt","w",encoding="utf8")

lines = [x.strip() for x in f_input.readlines()]
for line in lines:
    count+=1   
    #first line is a headerline. Skip intentionally.
    if count == 1:    
        continue

    lineTokens = line.split(',')
    analysis = str(TextBlob(clean_tweet(str(lineTokens[0]))))
    res=SentiStrength(analysis)
    if res == "false":
        print(lineTokens)
        continue

    res = [x.strip() for x in res.split(" ")]
    try:
        if int(res[-1]) == -1:
            sent = res[-2]
        else:
            sent = res[-3]
    except ValueError:
        print(lineTokens)
        continue

    print(lineTokens[0] + "\t" + str(int(sent)))
    f_output.write(lineTokens[0] + "\t" + str(int(sent)) + "\n")
    
f_output.close()
f_input.close()

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다